# COMPONENT 1 - WEB SCRAPING MODULE

### IMPORTING THE LIBRARIES

In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

### EXTRACTING NEWS DATA FOR TWO WEEKS (Feb 1, 2021 to Feb 14, 2021)

In [ ]:
#To scrape the webpages from Feb 1 to Feb 7
num_list = [ "01", "02", "03", "04", "05", "06", "07"]

In [ ]:
#To scrape the webpages from Feb 8 to Feb 14
num_list = [ "08", "09", "10", "11", "12", "13", "14"]

In [ ]:
num = 0
for i in range(len(num_list)):

  #List to store all the urls of each day
  URL = []

  page = requests.get("https://www.thehindu.com/archive/web/2021/02/{0}/".format(num_list[i]))
  soup = BeautifulSoup(page.content, "html.parser")

  #Scraping all the tags conataining links
  for item in soup.find_all(attrs={'class':'archive-list'}):
    for link in item.find_all(href=True):
        href=link.get('href')
        URL.append(href)

  #Scraping content from each webpage link in the url list      
  for url in URL:
    news_data = {"text" : []}
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    for p in soup.find_all('p'):
      news_data["text"].append(p.text)

    #Creating an independent .json file for each webpage  
    with open('thehindu_feb_02_file_{0}.json'.format(num), 'w') as jsonfile:
      json.dump(news_data, jsonfile)
    num+=1

### CREATING A LIST OF JSON FILES

In [ ]:
json_files = []
for i in range(len(URL_list)):
  json_files.append('thehindu_feb_02_file_{0}.json'.format(i))

### STORING THE TEXT FROM EACH FILE IN A LIST

In [ ]:
raw_text = []
for js in json_files:
    with open(js) as json_file:
        json_text = json.load(json_file)
    raw_text.append(json_text['text'])

### SAVING THE RAW TEXT IN A SEPARATE .JSON FILE

In [ ]:
#To save the data from Feb 1 to Feb 7
with open('raw_text_01.json', 'w') as jsonfile:
    json.dump(raw_text, jsonfile)

In [ ]:
#To save the data from Feb 8 to Feb 14
with open('raw_text_02.json', 'w') as jsonfile:
    json.dump(raw_text, jsonfile)

## CREATING A DATAFRAME TO STORE THE FILE NAMES AND THE CORRESPONDING URLs

In [ ]:
num_list = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14"]

### CREATING A LIST OF ALL THE URLs

In [ ]:
URL_list = []
for i in range(len(num_list)):
  page = requests.get("https://www.thehindu.com/archive/web/2021/02/{0}/".format(num_list[i]))
  soup = BeautifulSoup(page.content, "html.parser")
  for item in soup.find_all(attrs={'class':'archive-list'}):
    for link in item.find_all(href=True):
        href=link.get('href')
        URL_list.append(href)

In [ ]:
len(URL_list)

5554

In [ ]:
tab_head = ["File_Scraped", "Corresponding URL"]
df_data = pd.DataFrame(zip(json_files, URL_list), columns = tab_head)
df_data[:10]

,File_Scraped,Corresponding URL
0,thehindu_feb_02_file_0.json,https://www.thehindu.com/sport/other-sports/va...
1,thehindu_feb_02_file_1.json,https://www.thehindu.com/sport/tennis/advantag...
2,thehindu_feb_02_file_2.json,https://www.thehindu.com/sport/tennis/murray-r...
3,thehindu_feb_02_file_3.json,https://www.thehindu.com/sci-tech/technology/c...
4,thehindu_feb_02_file_4.json,https://www.thehindu.com/sci-tech/technology/e...
5,thehindu_feb_02_file_5.json,https://www.thehindu.com/sci-tech/technology/s...
6,thehindu_feb_02_file_6.json,https://www.thehindu.com/sci-tech/technology/e...
7,thehindu_feb_02_file_7.json,https://www.thehindu.com/sci-tech/technology/m...
8,thehindu_feb_02_file_8.json,https://www.thehindu.com/sci-tech/technology/a...
9,thehindu_feb_02_file_9.json,https://www.thehindu.com/sci-tech/technology/t...


### STORING THE DATAFRAME IN .CSV FILE

In [ ]:
df_data.to_csv('URLs_to_file_mapping.csv', sep ='|')

### DISPLAYING THE CONTENT IN .CSV FILE

In [ ]:
new_df = pd.read_csv('URLs_to_file_mapping.csv')
new_df.head(10)

,|File_Scraped|Corresponding URL
0,0|thehindu_feb_02_file_0.json|https://www.theh...
1,1|thehindu_feb_02_file_1.json|https://www.theh...
2,2|thehindu_feb_02_file_2.json|https://www.theh...
3,3|thehindu_feb_02_file_3.json|https://www.theh...
4,4|thehindu_feb_02_file_4.json|https://www.theh...
5,5|thehindu_feb_02_file_5.json|https://www.theh...
6,6|thehindu_feb_02_file_6.json|https://www.theh...
7,7|thehindu_feb_02_file_7.json|https://www.theh...
8,8|thehindu_feb_02_file_8.json|https://www.theh...
9,9|thehindu_feb_02_file_9.json|https://www.theh...


### VIEWING THE RAW DATA

In [ ]:
raw_text = []
f1 = open('raw_text_01.json')
data1 = json.load(f1)
for i in data1:
  raw_text.append(i) 

f2 = open('raw_text_02.json')
data2 = json.load(f2)
for i in data2:
  raw_text.append(i)

In [ ]:
len(raw_text)

5565